In [45]:
import os 
import pandas as pd 
import numpy as np 
import glob
import time

In [53]:
# file_path=fubar_logs[0]
# with open(file_path, 'r') as file: 
#     file_contents=file.readlines()
#     for row in file_contents:
#         if row.find('Loaded a multiple') != -1:
#             codons=row
            
codons.split("**")[3]

'1566'

In [62]:
#first, a function that determines line number bounds of the hyphy fubar results 

def number_hunter(file_path):
    with open(file_path, 'r') as file: 
        file_contents=file.readlines()
        for row in file_contents: 
            if row.find('Empty File') != -1:
                print('Error, empty file. Probably no tree file.')
                start=np.nan
                end=np.nan
                codon_len=np.nan
            word='Tabulating'
            if row.find(word) != -1:
                start=file_contents.index(row)
            if row.find('## FUBAR inferred') != -1:
                end=file_contents.index(row)
            if row.find('Loaded a multiple') != -1:
                codon_len=row.split("**")[3]
            
    return(start, end, codon_len)

index=number_hunter(fubar_out)

#then a function that converts log file to df, saves as csv in case I want it later, then returns a positions df 

def file_convert(start, end, codon_len, file_path):
    positions=pd.DataFrame()
    clade=file_path.split('/')[-2]
    if np.isnan(start) == True:
        positions=positions #return a blank df if no tree file 
    else: 
        with open(file_path, 'r') as file: 
            file_contents=file.readlines()
            for l_no, line in enumerate(file_contents[start+3:end-1]): #remove column names, and trailing data 
                codon=line.split('|')[1].strip('      ')
                partition=line.split('|')[2].strip('      ')
                alpha=line.split('|')[3].strip('      ')
                beta=line.split('|')[4].strip('      ')
                posterior_prob=line.split('|')[5].strip('      ').strip('Pos. posterior = ')
                row_res={'clade':clade, 'codon_len':int(codon_len), 'codon':int(codon), 'parititon':int(partition), 'alpha':float(alpha), 'beta':float(beta), 'posterior_prob':float(posterior_prob)}
                positions=positions.append(row_res, ignore_index=True)

    output_csv=file_path.strip('hyphy_fubar.log')+'hyphy_fubar_pos.csv'
    positions.to_csv(output_csv)
    return positions 
    
positions=file_convert(index[0], index[1], index[2], fubar_out)

In [46]:
#gather all fubar results files 
fubar_logs=glob.glob(os.path.join('/global/scratch/users/chandlersutherland/e14/popgen/clades/*/hyphy_fubar.log'))
len(fubar_logs)

212

In [63]:
num_clades=len(fubar_logs)
def wrapper(i):
    file_path=fubar_logs[i]
    index=number_hunter(file_path)
    positions=file_convert(index[0], index[1], index[2], file_path)
    return positions 

In [64]:
results=pd.DataFrame()
for i in range(0,len(fubar_logs)):
    res=wrapper(i)
    results=results.append(res)

Error, empty file. Probably no tree file.
Error, empty file. Probably no tree file.
Error, empty file. Probably no tree file.
Error, empty file. Probably no tree file.
Error, empty file. Probably no tree file.
Error, empty file. Probably no tree file.
Error, empty file. Probably no tree file.
Error, empty file. Probably no tree file.
Error, empty file. Probably no tree file.
Error, empty file. Probably no tree file.
Error, empty file. Probably no tree file.
Error, empty file. Probably no tree file.
Error, empty file. Probably no tree file.
Error, empty file. Probably no tree file.
Error, empty file. Probably no tree file.
Error, empty file. Probably no tree file.
Error, empty file. Probably no tree file.
Error, empty file. Probably no tree file.


In [66]:
#get number of sites with posterior prob > 90, 95, and 99 by clade 
fubar_95 = results.groupby(['clade', 'codon_len'])['posterior_prob'].apply(lambda x:(x>0.95).sum()).reset_index().rename(columns={'posterior_prob':'fubar_count_95'})
fubar_90 = results.groupby('clade')['posterior_prob'].apply(lambda x:(x>0.90).sum()).reset_index().rename(columns={'posterior_prob':'fubar_count_90'})
fubar_99 = results.groupby('clade')['posterior_prob'].apply(lambda x:(x>0.99).sum()).reset_index().rename(columns={'posterior_prob':'fubar_count_99'})

fubar_pos_counts=pd.merge(pd.merge(fubar_95, fubar_90, on='clade'), fubar_99, on='clade')
#fubar_pos_counts
fubar_pos_counts.to_csv('/global/scratch/users/chandlersutherland/e14/popgen/fubar_sites.csv')